In [ ]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, RobertaModel
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, classification_report
from spellchecker import SpellChecker
from collections import Counter
from datasets import Dataset
import string

In [ ]:
#!pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.8 MB/s eta 0:00:00


In [ ]:
#!pip install pyarrow datasets transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
#!pip install transformers==4.30.0 tokenizers==0.13.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.3
    Uninstalling transformers-4.43.3:
      Successfully uninstalled transformers-4.43.3


In [ ]:
#!pip install --upgrade pyarrow

In [ ]:
#!pip install accelerate

In [ ]:
#!pip uninstall -y transformers accelerate

In [ ]:
#!pip install transformers[torch] accelerate -U

In [ ]:
#!nvidia-smi


In [ ]:
# Uploading the trainset
uploaded = files.upload ()

Saving isear-train.xlsx to isear-train (1).xlsx


In [ ]:
# Loading the training file
custom_headers = ['Emotions', 'Text']
df = pd.read_excel('isear-train.xlsx',skiprows=1, header=None, names =custom_headers)

In [ ]:
# Encoding labels
label_encoding = {'anger': 0, 'disgust': 1, 'fear': 2, 'guilt': 3, 'joy': 4, 'sadness': 5, 'shame': 6}
y = df['Emotions'].values
y_train_encoded = np.array([label_encoding[label] for label in y])
y_train_tensor = torch.tensor (y_train_encoded)

print("Encoded labels:", y, y_train_encoded[:20])

Encoded labels: ['sadness' 'disgust' 'fear' ... 'sadness' 'disgust' 'fear'] [5 1 2 6 0 6 1 4 6 4 1 0 0 4 2 4 6 2 1 5]


In [ ]:
# Trainset preprocessing

def tokenize(text):
    translation_table = str.maketrans({c: f' {c} ' if not c.isalnum() else c for c in set(text)})  # creates translation
    # table(dictionary) with the built-in function maketrans, set(text) makes an unordered collection of unique
    # elements through set comprehension, a concise way to create sets
    tokenized_text = text.translate(translation_table)  # uses translation table to add whitespace around special
    # characters and punctuation
    return tokenized_text.strip().lower().split()  # split-tokens split on space, lower-tokens made all lowercase,
    # strip-and leading or trailing whitespaces are removed from string

# Join sentences
text = ''.join(df['Text'].astype(str))
#print (text)

tokenized_text = []
for index, sentence in enumerate(df['Text']):
    emotion_label = df['Emotions'][index]
    sentence = tokenize(sentence)
    tokenized_text.append (sentence)

# Filter out punctuations
filtered_tokens = []

for tokens in tokenized_text:
    clean_tokens = []

    for token in tokens:
        if all(char in string.punctuation for char in token):
            continue  # Skip punctuation tokens
        if token.isdigit():
            continue  # Skip digit tokens
        else:
            clean_tokens.append(token)

    filtered_tokens.append(clean_tokens)

tokenized_text = filtered_tokens
#print (filtered_tokens)

In [ ]:
# Correct misspelled words

# Initialize the spell checker
spell = SpellChecker()

# Flatten the list of lists
just_tokens = []
for sentence in filtered_tokens:
    for token in sentence:
        just_tokens.append (token)


# Count the frequency of each token
token_counts = Counter(just_tokens)

# Find tokens that appear only once
uncommon_words = [token for token in token_counts if token_counts[token] == 1]

#print (len (uncommon_words))

# Check if the uncommen words were misspelled
misspelled_words = spell.unknown(uncommon_words)
#print (misspelled_words)

# Create a dictionary of misspelled words and their corrections
corrections = {word: spell.correction(word) for word in misspelled_words}
#print("Misspelled words and corrections:", corrections)

In [ ]:
# Apply corrections to the text

def correct_spelling(text, corrections):
    corrected_text = []
    for sentence in text: # Correct each token in the sentence
        corrected_tokens = [corrections.get(token, token) if corrections.get(token, token) else token for token in sentence]  # Correct the tokens
        corrected_sentence = ' '.join(corrected_tokens)  # Join the tokens back into a string (sentence)
        corrected_text.append(corrected_sentence)
    return corrected_text

# Apply spell correction to each sublist of sentences in the list of lists
corrected_texts = [correct_spelling([sentence], corrections) for sentence in filtered_tokens]
print (len (corrected_texts))


df['Corrected_Text'] = corrected_texts
#print (df['Corrected_Text'])


5366


In [ ]:
# Rename the headers for trainset
custom_headers = ['Emotions', 'Corrected_Text']
df = pd.read_excel('isear-train.xlsx', skiprows=1, header=None, names=custom_headers)

corr_strings = []
for index, sentence in enumerate(df['Corrected_Text']):
    emotion_label = df['Emotions'][index]
    corr_strings.append (sentence)


In [ ]:
# Uploading testset
from google.colab import files
uploaded = files.upload ()

Saving isear-test.xlsx to isear-test.xlsx


In [ ]:
# Loading the testset
custom_headers_test = ['Emotions_test', 'Text_test']
df_test = pd.read_excel("isear-test.xlsx", skiprows=1, header=None, names=custom_headers_test)

# Data and label processing for testset
strings_test = []
for index, sentence in enumerate(df_test['Text_test']):
    emotion_label_test = df_test['Emotions_test'][index]
    strings_test.append (sentence)

strings_test = df_test['Text_test'].tolist()
print (strings_test[0:3])

# Label encoding
y_test = df_test['Emotions_test'].values
y_test_encoded = np.array([label_encoding[label] for label in y_test])
y_test_tensor = torch.tensor (y_test_encoded)
print (len (df_test['Text_test']))

['My first feeling of falling in love.  Coming across a person who made me feel love, happiness, joy, giving me a new and beautiful future, who made me dream.', 'When I saw that the mark I had obtained in an exam was a good one.', 'When I learnt that I had been admitted to the university.']
1150


In [ ]:
# Checking misspelled words in the testset
# Convert the sentences to a single string
text_test = ' '.join(df_test['Text_test'].astype(str))

# Tokenize the testset
tokenized_text_test = []
for index, sentence in enumerate(df_test['Text_test']):
    emotion_label = df_test['Emotions_test'][index]
    tokenized_sentence = tokenize(sentence)
    tokenized_text_test.append(tokenized_sentence)

filtered_tokens_test = []

# Filter out punctuation and digit tokens
for tokens in tokenized_text_test:
    clean_tokens = []
    for token in tokens:
        if all(char in string.punctuation for char in token):
            continue  # Skip punctuation tokens
        if token.isdigit():
            continue  # Skip digit tokens
        else:
            clean_tokens.append(token)
    filtered_tokens_test.append(clean_tokens)

print (filtered_tokens_test)

[['my', 'first', 'feeling', 'of', 'falling', 'in', 'love', 'coming', 'across', 'a', 'person', 'who', 'made', 'me', 'feel', 'love', 'happiness', 'joy', 'giving', 'me', 'a', 'new', 'and', 'beautiful', 'future', 'who', 'made', 'me', 'dream'], ['when', 'i', 'saw', 'that', 'the', 'mark', 'i', 'had', 'obtained', 'in', 'an', 'exam', 'was', 'a', 'good', 'one'], ['when', 'i', 'learnt', 'that', 'i', 'had', 'been', 'admitted', 'to', 'the', 'university'], ['in', 'a', 'hurry', 'to', 'get', 'to', 'lectures', 'and', 'the', 'car', 'would', 'not', 'start'], ['not', 'attending', 'school', 'when', 'am', 'not', 'sick', 'but', 'in', 'need', 'of', 'mental', 'rest', 'staff', 'on', 'your', 'return', 'query', 'your', 'sickness', 'it', 'is', 'easier', 'á'], ['when', 'i', 'went', 'up', 'the', 'stairs', 'in', 'a', 'high', 'dark', 'house', 'full', 'of', 'shadows', 'passed', 'a', 'door', 'and', 'had', 'the', 'feeling', 'that', 'someone', 'or', 'something', 'was', 'behind', 'me', 'although', 'i', 'knew', 'i', 'was',

In [ ]:
# Initialize spellchecker
spell = SpellChecker()

# Flatten the list of lists into a single list
just_tokens_test = [token for sentence in filtered_tokens_test for token in sentence]
#Count all tokens
token_counts_test = Counter(just_tokens_test)

# Find the tokens that appear only once
uncommon_words_test = [token for token in token_counts_test if token_counts_test[token] == 1]
#Check for misspellings
misspelled_words_test = spell.unknown(uncommon_words_test)

#print(f"Misspelled words: {misspelled_words_test}")
corrections_test = {word: spell.correction(word) for word in misspelled_words_test}
#print(f"Misspelled words and corrections: {corrections_test}")
#print (len(misspelled_words_test))

# Apply spelling corrections to each sentence in the text
corrected_texts_test = [correct_spelling([sentence], corrections_test) for sentence in filtered_tokens_test]
#print (len (corrected_texts_test))

corr_strings_test = df_test['Text_test']
corr_strings_test = [item for sublist in corrected_texts_test for item in sublist]
print (corr_strings_test)

['my first feeling of falling in love coming across a person who made me feel love happiness joy giving me a new and beautiful future who made me dream', 'when i saw that the mark i had obtained in an exam was a good one', 'when i learnt that i had been admitted to the university', 'in a hurry to get to lectures and the car would not start', 'not attending school when am not sick but in need of mental rest staff on your return query your sickness it is easier á', 'when i went up the stairs in a high dark house full of shadows passed a door and had the feeling that someone or something was behind me although i knew i was alone', 'at my grandfather s funeral i remembered that the thing he liked and respected most was earth i thought that he had attained what he wanted and was not separated from it any longer', 'when i lost the notes of the lecturer i had borrowed from my friend', 'my friend was so intoxicated that the police came and took him away and they later contacted his parents', '

In [ ]:
# Uploading the devset
from google.colab import files
uploaded = files.upload ()

Saving isear-validation.xlsx to isear-validation.xlsx


In [ ]:
# Loading Devset
custom_headers_dev = ['Emotions_dev', 'Text_dev']
df_dev = pd.read_excel("isear-validation.xlsx", skiprows=1, header=None, names=custom_headers_dev)
print (type(df_dev['Text_dev']))
strings_dev = []
for index, sentence in enumerate(df_dev['Text_dev']):
    emotion_label_dev = df_dev['Emotions_dev'][index]
    strings_dev.append (sentence)
print (strings_dev[0:3])

# Label encoding for devset
y_dev = df_dev['Emotions_dev'].values
y_dev_encoded = np.array([label_encoding[label] for label in y_dev])
y_dev_tensor = torch.tensor (y_dev_encoded)
#print (len (strings_dev))
#print (y_dev_tensor.shape)


<class 'pandas.core.series.Series'>
['[ Laziness makes one sad.]', 'I had to leave my girlfriend for 4 weeks because of my studies. This happens frequently. The last 24 hours before my departure are always very depressing.', 'I promised to join a meeting but did not turn up as I did not want to do the work.']


In [ ]:
# Checking misspelled words in the testset
# Convert the sentences to a single string
text_test = ' '.join(df_dev['Text_dev'].astype(str))

# Tokenize the testset
tokenized_text_dev = []
for index, sentence in enumerate(df_dev['Text_dev']):
    emotion_label = df_dev['Emotions_dev'][index]
    tokenized_sentence = tokenize(sentence)
    tokenized_text_dev.append(tokenized_sentence)

filtered_tokens_dev = []

# Filter out punctuation and digit tokens
for tokens in tokenized_text_dev:
    clean_tokens = []
    for token in tokens:
        if all(char in string.punctuation for char in token):
            continue  # Skip punctuation tokens
        if token.isdigit():
            continue  # Skip digit tokens
        else:
            clean_tokens.append(token)
    filtered_tokens_dev.append(clean_tokens)

print (filtered_tokens_dev)

[['laziness', 'makes', 'one', 'sad'], ['i', 'had', 'to', 'leave', 'my', 'girlfriend', 'for', 'weeks', 'because', 'of', 'my', 'studies', 'this', 'happens', 'frequently', 'the', 'last', 'hours', 'before', 'my', 'departure', 'are', 'always', 'very', 'depressing'], ['i', 'promised', 'to', 'join', 'a', 'meeting', 'but', 'did', 'not', 'turn', 'up', 'as', 'i', 'did', 'not', 'want', 'to', 'do', 'the', 'work'], ['hearing', 'fran', 'wilde', 'introduce', 'the', 'bill'], ['when', 'the', 'person', 'i', 'am', 'going', 'out', 'with', 'let', 'me', 'down', 'in', 'a', 'situation', 'that', 'i', 'was', 'depending', 'on', 'them', 'and', 'didn', 't', 'show', 'respect', 'or', 'care'], ['at', 'an', 'exam', 'in', 'pharmacology', 'i', 'tried', 'to', 'prompt', 'a', 'coleague', 'she', 'asked', 'me', 'something', 'and', 'the', 'assistant', 'professor', 'made', 'me', 'a', 'sharp', 'remark'], ['reading', 'a', 'letter', 'from', 'a', 'close', 'friend', 'who', 'said', 'he', 'missed', 'me', 'and', 'cared', 'for', 'me'],

In [ ]:
# Initialize spellchecker
spell = SpellChecker()

# Flatten the list of lists into a single list
just_tokens_dev = [token for sentence in filtered_tokens_dev for token in sentence]
#Count all tokens
token_counts_dev = Counter(just_tokens_dev)

# Find the tokens that appear only once
uncommon_words_dev = [token for token in token_counts_dev if token_counts_dev[token] == 1]
#Check for misspellings
misspelled_words_dev = spell.unknown(uncommon_words_dev)

#print(f"Misspelled words: {misspelled_words_test}")
corrections_dev = {word: spell.correction(word) for word in misspelled_words_dev}
#print(f"Misspelled words and corrections: {corrections_test}")
#print (len(misspelled_words_test))

# Apply spelling corrections to each sentence in the text
corrected_texts_dev = [correct_spelling([sentence], corrections_dev) for sentence in filtered_tokens_dev]
#print (len (corrected_texts_test))

corr_strings_dev = df_dev['Text_dev']
corr_strings_dev = [item for sublist in corrected_texts_dev for item in sublist]
print (corr_strings_dev)

['laziness makes one sad', 'i had to leave my girlfriend for weeks because of my studies this happens frequently the last hours before my departure are always very depressing', 'i promised to join a meeting but did not turn up as i did not want to do the work', 'hearing ran wild introduce the bill', 'when the person i am going out with let me down in a situation that i was depending on them and didn t show respect or care', 'at an exam in pharmacology i tried to prompt a colleague she asked me something and the assistant professor made me a sharp remark', 'reading a letter from a close friend who said he missed me and cared for me', 'when i secretly tried smoking for the first time being years old', 'my sister borrowed some up records for a party she had left them in the car on curaçao due to the heat they were crooked so i couldn t play them again', 'i was to take a test and i had not made the necessary preparations until it was too late there were two days to go and i still had not d

In [ ]:
# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize function with progress print
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

# Prepare the data for Dataset object
train_data = {'text': corr_strings, 'labels': y_train_encoded.tolist()}
val_data = {'text': corr_strings_dev, 'labels': y_dev_encoded.tolist()}
test_data = {'text': corr_strings_test, 'labels': y_test_encoded.tolist()}

# Create Dataset objects
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

# Tokenize the data
print("Tokenizing train dataset...")
train_dataset = train_dataset.map(tokenize_function, batched=True)
print("Tokenizing validation dataset...")
val_dataset = val_dataset.map(tokenize_function, batched=True)
print("Tokenizing test dataset...")
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Map:   0%|          | 0/5366 [00:00<?, ? examples/s]

Tokenizing validation dataset...


Map:   0%|          | 0/1150 [00:00<?, ? examples/s]

Tokenizing test dataset...


Map:   0%|          | 0/1150 [00:00<?, ? examples/s]

In [ ]:
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Initialize the model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoding))

if torch.cuda.is_available():
    model.to('cuda')
else:
    print("CUDA is not available. Using CPU.")


# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_macro = f1_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='weighted')
    return {"f1": f1, "f1_macro": f1_macro}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate(test_dataset)
print(results)


results_val = trainer.evaluate(val_dataset)
print(results_val)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:  50%|#####     | 252M/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should pr

Epoch,Training Loss,Validation Loss,F1,F1 Macro
1,1.368200,0.877630,0.689042,0.685966
2,0.831900,0.872302,0.706897,0.703404
3,0.539800,0.847574,0.735041,0.732001


{'eval_loss': 0.9117454886436462, 'eval_f1': 0.7127178652912566, 'eval_f1_macro': 0.7161716151114682, 'eval_runtime': 34.1785, 'eval_samples_per_second': 33.647, 'eval_steps_per_second': 4.213, 'epoch': 3.0}
{'eval_loss': 0.8475738763809204, 'eval_f1': 0.7350406106285466, 'eval_f1_macro': 0.7320010342127781, 'eval_runtime': 34.5471, 'eval_samples_per_second': 33.288, 'eval_steps_per_second': 4.168, 'epoch': 3.0}


In [ ]:
"""# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_per_class = f1_score(labels, predictions, average=None, labels=list(label_encoding.values()))
    report = classification_report(labels, predictions, target_names=label_encoding.keys(), digits=4)
    print(report)
    return {"f1_weighted": f1_weighted, "f1_macro": f1_macro, "f1_per_class": f1_per_class}"""

In [ ]:

# Get predictions from the model
predictions = trainer.predict(test_dataset)

# Extract the predicted and true labels
preds = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

# Print classification report
target_names = list(label_encoding.keys())
report = classification_report(true_labels, preds, target_names=target_names)
print(report)

              precision    recall  f1-score   support

       anger       0.66      0.61      0.64       166
     disgust       0.72      0.66      0.69       166
        fear       0.81      0.81      0.81       168
       guilt       0.61      0.70      0.65       155
         joy       0.92      0.89      0.91       152
     sadness       0.69      0.78      0.73       161
       shame       0.61      0.57      0.59       182

    accuracy                           0.71      1150
   macro avg       0.72      0.72      0.72      1150
weighted avg       0.72      0.71      0.71      1150



In [ ]:
# Get predictions
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = y_test_encoded

# Identify mislabeled sentences
mislabeled_indices = np.where(predicted_labels != true_labels)[0]
mislabeled_sentences = [test_data['text'][i] for i in mislabeled_indices]
mislabeled_true_labels = [true_labels[i] for i in mislabeled_indices]
mislabeled_pred_labels = [predicted_labels[i] for i in mislabeled_indices]

# Display mislabeled sentences with their true and predicted labels
for sentence, true_label, pred_label in zip(mislabeled_sentences, mislabeled_true_labels, mislabeled_pred_labels):
    print(f"Sentence: {sentence}")
    print(f"True Label: {true_label}, Predicted Label: {pred_label}")
    print()

Sentence: in a hurry to get to lectures and the car would not start
True Label: 0, Predicted Label: 2

Sentence: when i lost the notes of the lecturer i had borrowed from my friend
True Label: 1, Predicted Label: 0

Sentence: my friend was so intoxicated that the police came and took him away and they later contacted his parents
True Label: 6, Predicted Label: 2

Sentence: i did not quite succeed in breast feeding my baby
True Label: 5, Predicted Label: 3

Sentence: the fact that i hadn t been informed about something and i had the greatest trust in this person
True Label: 0, Predicted Label: 5

Sentence: i went out dancing and found my boyfriend kissing another girl
True Label: 0, Predicted Label: 1

Sentence: when someone whom i considered my friend without telling me he was annoyed proceeded to ignore me for several meets with no apparent reason
True Label: 1, Predicted Label: 0

Sentence: when at school i thought that i was going to get the best results in mathematics but it was so